In [76]:
import os
import pandas as pd
import tqdm.notebook as tq

In [77]:
folder_path = os.getcwd()
 
# 获取当前文件夹下所有文件名
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
 
person_names=[name.split('.')[0] for name in file_names]

In [78]:
eye_root_dir='/data/guohua/BeiJing/data/new_UAV123/UAV123/'
data_dir="/data/guohua/BeiJing/data/UAV123/UAV_Lasot_struc"

In [72]:
# 填充值
fill_value = '-1'

# key columns
key_columns = [
    'Participant name',
    'Gaze point X',
    'Gaze point Y',
    'Gaze point left X',
    'Gaze point left Y',
    'Gaze point right X',
    'Gaze point right Y',
    'Gaze direction left X',
    'Gaze direction left Y',
    'Gaze direction left Z',
    'Gaze direction right X',
    'Gaze direction right Y',
    'Gaze direction right Z',
    'Pupil diameter left',
    'Pupil diameter right',
    'Pupil diameter filtered',
    'Eye openness left',
    'Eye openness right',
    'Eye openness filtered',
    'Validity left',
    'Validity right',
    'Eye position left X (DACSmm)',
    'Eye position left Y (DACSmm)',
    'Eye position left Z (DACSmm)',
    'Eye position right X (DACSmm)',
    'Eye position right Y (DACSmm)',
    'Eye position right Z (DACSmm)',
    'Gaze point left X (DACSmm)',
    'Gaze point left Y (DACSmm)',
    'Gaze point right X (DACSmm)',
    'Gaze point right Y (DACSmm)',
    'Gaze point X (MCSnorm)',
    'Gaze point Y (MCSnorm)',
    'Gaze point left X (MCSnorm)',
    'Gaze point left Y (MCSnorm)',
    'Gaze point right X (MCSnorm)',
    'Gaze point right Y (MCSnorm)',
    'Presented Stimulus name',
    'Presented Media name',
    'Presented Media width',
    'Presented Media height',
    'Presented Media position X (DACSpx)',
    'Presented Media position Y (DACSpx)',
    'Eye movement type',
    'Gaze event duration',
    'Eye movement type index',
    'Fixation point X',
    'Fixation point Y',
    'Fixation point X (MCSnorm)',
    'Fixation point Y (MCSnorm)',
]
# misc 列 表示一些混杂数据 单独存放
misc_columns = [
    'Recording timestamp',
    'Computer timestamp',
    'Sensor',
    'Project name',
    'Export date',
    'Recording name',
    'Recording date',
    'Recording date UTC',
    'Recording start time',
    'Recording start time UTC',
    'Recording duration',
    'Timeline name',
    'Recording Fixation filter name',
    'Recording software version',
    'Recording resolution height',
    'Recording resolution width',
    'Recording monitor latency',
    'Average calibration accuracy (mm)',
    'Average calibration precision SD (mm)',
    'Average calibration precision RMS (mm)',
    'Average calibration accuracy (degrees)',
    'Average calibration precision SD (degrees)',
    'Average calibration precision RMS (degrees)',
    'Average calibration accuracy (pixels)',
    'Average calibration precision SD (pixels)',
    'Average calibration precision RMS (pixels)',
    'Average validation accuracy (mm)',
    'Average validation precision SD (mm)',
    'Average validation precision RMS (mm)',
    'Average validation accuracy (degrees)',
    'Average validation precision SD (degrees)',
    'Average validation precision RMS (degrees)',
    'Average validation accuracy (pixels)',
    'Average validation precision SD (pixels)',
    'Average validation precision RMS (pixels)',
    'Eyetracker timestamp',
    'Event',
    'Event value',
]

In [73]:
# 打开 txt 文件，将内容读取为列表
with open('/data/guohua/BeiJing/data/UAV123/new_seq_names.txt', 'r') as file:
    content = file.read()  # 读取整个文件的内容为一个字符串

# 去除结尾的多余逗号（如果有的话），并将字符串按逗号分割为列表
items = content.strip().split(',')

# 去除每个元素的多余空格或换行符，以及两侧的引号
clazz_list = [item.strip().strip('"') for item in items]

In [ ]:
for person_name in tq.tqdm(person_names,desc='person'):
    path=os.path.join(eye_root_dir, f'{person_name}.tsv')
    data=pd.read_csv(path, sep='\t')
    data['Presented Stimulus name'] = data['Presented Stimulus name'].fillna('')

    # 使用布尔索引筛选出不包含 '0000001' 的行
    filtered_data = data[~data['Presented Stimulus name'].str.contains('0000001')]
    data = data.fillna(fill_value)
    data['Presented Stimulus name'] = data['Presented Stimulus name'].fillna('').astype(str)
    data = data[~data['Presented Stimulus name'].str.contains('00000')]


    groups = data.groupby('Presented Stimulus name')

    # 创建目标目录，如果不存在则新建
    output_dir = os.path.join("../UAV123-prossesed", person_name)
    os.makedirs(output_dir, exist_ok=True)

    # 使用 tqdm 显示进度条
    for group_name, group_df in tq.tqdm(groups, desc="Saving groups to TSV files"):
        # 为每个类别创建一个新的 TSV 文件，并保存到指定路径
        file_name = os.path.join(output_dir, f"{group_name}.tsv")
        group_df.to_csv(file_name, sep='\t', index=False)
        # tq.tqdm.write(f"Saved group {group_name} to file {file_name}")
    

    for clazz in clazz_list:  
        # load
        eye_raw_data = pd.read_csv(os.path.join(output_dir, f"{clazz}.tsv"), sep='\t')

        eye_raw_key_data = eye_raw_data[key_columns].copy()  # 使用 .copy() 创建一个副本
        # 增加一列 'is_valid' 'is_terminal' 并指定默认值为 0
        eye_raw_key_data.loc[:,'is_valid'] = 0 # 表示本条眼动数据是否有效
        eye_raw_key_data.loc[:,'is_frame_terminal'] = 0 # 表示一帧视频对应的最后一个眼动数据
        eye_raw_misc_data = eye_raw_data[misc_columns]
        
        presented_stimulus_name = eye_raw_data['Presented Stimulus name'].to_list()
        
        result_dir = os.path.join("../UAV123-preprocessed", person_name, clazz)
        os.makedirs(result_dir, exist_ok=True)
        # init
        sample_key_data = []
        sample_misc_data = []
        sample_name = None
        for idx, name in tq.tqdm(enumerate(presented_stimulus_name), desc=f'{clazz}'):
            if name == fill_value or name == 'Eyetracker Calibration' or name == 'restImage': 
                continue
            
            if name != sample_name or idx == len(presented_stimulus_name)-1: 
                # save
                if len(sample_key_data) != 0:
                    sample_key_data = pd.concat(sample_key_data)
                    sample_key_data.to_csv(os.path.join(result_dir, f"{sample_name}.tsv"), sep='\t', index=False)
                    sample_misc_data = pd.concat(sample_misc_data)
                    sample_misc_data.to_csv(os.path.join(result_dir, f"{sample_name}_misc.tsv"), sep='\t', index=False)
                # init
                sample_name = name
                sample_key_data = []
                sample_misc_data = []
                frame_start_idx = idx # 0-based
                frame_eye_sample_num = int(sum(eye_raw_data['Presented Stimulus name'] == sample_name) / len(os.listdir(os.path.join(data_dir, clazz, sample_name, 'img'))))
            
            key_data_slice = eye_raw_key_data.iloc[[idx], :]
            misc_data_slice = eye_raw_misc_data.iloc[[idx], :]
            
            # check valid
            if (key_data_slice != fill_value).all(axis=1).values.all():
                if key_data_slice['Validity left'].item() == 'Valid' and key_data_slice['Validity right'].item() == 'Valid':
                    key_data_slice.loc[:,'is_valid'] = 1
                    pass
            # check frame terminal
            if frame_start_idx + frame_eye_sample_num - 1 == idx:
                frame_start_idx = idx + 1 # 0-based
                key_data_slice.loc[:,'is_frame_terminal'] = 1
                pass

            sample_key_data.append(key_data_slice)
            sample_misc_data.append(misc_data_slice)

person:   0%|          | 0/17 [00:00<?, ?it/s]

/tmp/ipykernel_904376/4268128985.py:3: DtypeWarning: Columns (37,38,76) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(path, sep='\t')


Saving groups to TSV files:   0%|          | 0/94 [00:00<?, ?it/s]

Saved group  to file ../UAV123-prossesed/renwenhao/.tsv
Saved group Eyetracker Calibration to file ../UAV123-prossesed/renwenhao/Eyetracker Calibration.tsv
Saved group bike1 to file ../UAV123-prossesed/renwenhao/bike1.tsv
Saved group bike2 to file ../UAV123-prossesed/renwenhao/bike2.tsv
Saved group bike3 to file ../UAV123-prossesed/renwenhao/bike3.tsv
Saved group bird1 to file ../UAV123-prossesed/renwenhao/bird1.tsv
Saved group boat1 to file ../UAV123-prossesed/renwenhao/boat1.tsv
Saved group boat2 to file ../UAV123-prossesed/renwenhao/boat2.tsv
Saved group boat3 to file ../UAV123-prossesed/renwenhao/boat3.tsv
Saved group boat4 to file ../UAV123-prossesed/renwenhao/boat4.tsv
Saved group boat5 to file ../UAV123-prossesed/renwenhao/boat5.tsv
Saved group boat6 to file ../UAV123-prossesed/renwenhao/boat6.tsv
Saved group boat7 to file ../UAV123-prossesed/renwenhao/boat7.tsv
Saved group boat8 to file ../UAV123-prossesed/renwenhao/boat8.tsv
Saved group boat9 to file ../UAV123-prossesed/renwen

bike1: 0it [00:00, ?it/s]

In [47]:
# import re
# filtered_items = [item for item in clazz_list if not re.search(r'\d+_\d+', item)]
# other_items=[item for item in clazz_list if re.search(r'\d+_\d+', item)]

In [53]:
import pandas as pd
import numpy as np
import os
import tqdm.notebook as tq

In [54]:
root_dir = '/data/guohua/BeiJing/data/new_UAV123/UAV123-preprocessed/fanyanlong'
lasot_dir = '/data/guohua/BeiJing/data/UAV123/UAV_Lasot_struc'
output_dir = '/data/guohua/BeiJing/data/UAV-eye-sy/fanyanlong'
os.makedirs(output_dir, exist_ok=True)


In [55]:
for clazz in clazz_list:
    clazz_dir = os.path.join(root_dir, clazz)
    sample_list = os.listdir(clazz_dir)
    for sample in tq.tqdm(sample_list, desc=f'{clazz}'):
        # 跳过misc
        if sample.endswith('_misc.tsv'):
            continue
        
        sample_name = sample.replace('.tsv', '')
        sample_path = os.path.join(clazz_dir, sample)
        eye_data = pd.read_csv(sample_path, sep='\t')
        video_frame_num = len(pd.read_csv(os.path.join(lasot_dir, clazz, sample_name, 'groundtruth.txt'), delimiter=',', header=None, dtype=np.int64, na_filter=False).values)

        video_frame_eye = []
        final_result = []
        
        for eye_index in range(len(eye_data['is_frame_terminal'])):
            is_frame_terminal = eye_data['is_frame_terminal'][eye_index]
            is_valid = eye_data['is_valid'][eye_index]
            
            # 跳过无效数据
            if is_valid == 1:
                w = eye_data['Presented Media width'][eye_index]
                h = eye_data['Presented Media height'][eye_index]
                x = eye_data['Gaze point X (MCSnorm)'][eye_index]
                y = eye_data['Gaze point Y (MCSnorm)'][eye_index]
                video_frame_eye.append((int(x*w), int(y*h)))
            
            # 终止帧
            if is_frame_terminal == 1:
                # 判断1个video帧内的眼动数据是否都是无效，若是赋值为(0, 0)
                if len(video_frame_eye) == 0: video_frame_eye = np.array([0, 0], dtype=np.int64)
                else:
                    video_frame_eye = np.array(video_frame_eye)
                    video_frame_eye = np.mean(video_frame_eye, axis=0, dtype=np.int64)
                final_result.append(video_frame_eye)
                video_frame_eye = []
            
            # 跳过多余的眼动数据
            if len(final_result) == video_frame_num:
                break
        
        # 保存
        output_path = os.path.join(output_dir, clazz)
        os.makedirs(output_path, exist_ok=True)
        np.savetxt(os.path.join(output_path, f'{sample_name}.txt'), np.array(final_result), fmt='%d', delimiter=',')

bike1:   0%|          | 0/2 [00:00<?, ?it/s]

bike2:   0%|          | 0/2 [00:00<?, ?it/s]

bike3:   0%|          | 0/2 [00:00<?, ?it/s]

boat1:   0%|          | 0/2 [00:00<?, ?it/s]

boat2:   0%|          | 0/2 [00:00<?, ?it/s]

...

wakeboard5:   0%|          | 0/2 [00:00<?, ?it/s]

wakeboard6:   0%|          | 0/2 [00:00<?, ?it/s]

wakeboard7:   0%|          | 0/2 [00:00<?, ?it/s]

wakeboard8:   0%|          | 0/2 [00:00<?, ?it/s]

wakeboard9:   0%|          | 0/2 [00:00<?, ?it/s]

In [58]:
print(clazz_list)

['bike1', 'bike2', 'bike3', 'boat1', 'boat2', 'boat3', 'boat4', 'boat5', 'boat6', 'boat7', 'boat8', 'boat9', 'building1', 'building2', 'building3', 'building4', 'building5', 'car1', 'car10', 'car11', 'car12', 'car13', 'car14', 'car15', 'car16', 'car17', 'car18', 'car1_s', 'car2', 'car2_s', 'car3', 'car3_s', 'car4', 'car4_s', 'car5', 'car6', 'car7', 'car8', 'car9', 'group1', 'group2', 'group3', 'person1', 'person10', 'person11', 'person12', 'person13', 'person14', 'person15', 'person16', 'person17', 'person18', 'person19', 'person1_s', 'person2', 'person20', 'person21', 'person22', 'person23', 'person2_s', 'person3', 'person3_s', 'person4', 'person5', 'person6', 'person7', 'person8', 'person9', 'truck1', 'truck2', 'truck3', 'truck4', 'uav1', 'uav2', 'uav3', 'uav4', 'uav5', 'uav6', 'uav7', 'uav8', 'wakeboard1', 'wakeboard10', 'wakeboard2', 'wakeboard3', 'wakeboard4', 'wakeboard5', 'wakeboard6', 'wakeboard7', 'wakeboard8', 'wakeboard9']
